# Cross Validation

As nossas decisões não podem depender da aleatoriedade dos algoritmos.
Por exemplo validar um modelo de acordo com sua eficácia, sendo que a eficácia foi calculada utilizando um SEED aleatório.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_validate

In [2]:
url = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(url)

dados = dados.drop(columns=["Unnamed: 0"], axis = 1)

x = dados[['preco','idade_do_modelo','km_por_ano']]
y = dados['vendido']

SEED = 158020
np.random.seed(SEED)

print("SEED utilizado: %d" % SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print("Treino com %d elementos e teste com %d elementos." % (len(treino_x), len(teste_x)))

## Taxa de acerto baseline #######################################

dummy = DummyClassifier(strategy = 'stratified')
dummy.fit(treino_x, treino_y)
acuracia_dummy = dummy.score(teste_x, teste_y)

print("Acurácia Dummy: %.2f%%" % (acuracia_dummy * 100))

## Taxa de acerto Decision Tree ###################################

decisionTree = DecisionTreeClassifier(max_depth = 2)
decisionTree.fit(treino_x, treino_y)

previsoes = decisionTree.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes)

print("Acurácia Decision Tree com 2 níveis: %.2f%%" % (acuracia * 100))

SEED utilizado: 158020
Treino com 7500 elementos e teste com 2500 elementos.
Acurácia Dummy: 50.96%
Acurácia Decision Tree com 2 níveis: 71.92%


# KFold

Os valores de acurácia se dão por conta do valor de SEED passado. Queremos uma estimativa menos relacionada a aleatoriedade.

 - Um método é rodar o treino e o teste diversas vezes para encontrar um intervalo de estimativa, ao invés de apenas um número. Dessa forma, minimizamos a influência de agentes externos como a aleatoriedade, que pode nos dar um valor muito bom ou muito ruim, dependendo da sorte/azar.

In [3]:
## Cross validate:

def cross_validate_muda_params(seed = 1234, cv = 5):
    np.random.seed(seed)

    modelo = DecisionTreeClassifier(max_depth = 2)

    resultados = cross_validate(modelo, x, y, cv = cv) # cv -> em quantos pedaços a validação será quebrada

    resultados_teste = resultados['test_score']
    media_teste = resultados_teste.mean()
    desvio_padrao_teste = resultados_teste.std()
    intervalo_acuracia = [media_teste-(2*desvio_padrao_teste), media_teste+(2*desvio_padrao_teste)]  # [media - 2*desvio_padrao, media + 2*Desvio Padrao]

    print("Resultados parciais: ", resultados_teste)
    print("Média: %.2f%%" % (media_teste * 100))
    print("Intervalo de acurácia: %.2f%% ~ %.2f%%" % (intervalo_acuracia[0]*100, intervalo_acuracia[1]*100))

In [4]:
cross_validate_muda_params(1234, 5)

Resultados parciais:  [0.756  0.7565 0.7625 0.7545 0.7595]
Média: 75.78%
Intervalo de acurácia: 75.21% ~ 76.35%


In [5]:
cross_validate_muda_params(9876, 5)

Resultados parciais:  [0.756  0.7565 0.7625 0.7545 0.7595]
Média: 75.78%
Intervalo de acurácia: 75.21% ~ 76.35%


Dessa forma, mesmo mudando o SEED, obtemos o mesmo intervalo de eficácia.

Porém, ainda estamos suscetíveis a mudança por conta do número em que dividimos os dados (parâmetro cv do método cross_validate). Normalmente usa-se entre 5 e 10 cv, existem estudos científicos evidenciando esses valores.

In [6]:
cross_validate_muda_params(9876, 10)

Resultados parciais:  [0.742 0.77  0.749 0.764 0.761 0.764 0.754 0.755 0.759 0.76 ]
Média: 75.78%
Intervalo de acurácia: 74.24% ~ 77.32%


Por padrão, o algoritmo Cross Validate não embaralha os dados para fazer as divisões. Vamos implementar o embaralhamento utilizando o KFold:

In [7]:
from sklearn.model_selection import KFold

cv = KFold(n_splits = 10) # Sem embaralhamento

cross_validate_muda_params(9876, cv) ## agora passamos um cross validator como cv, ao invés de um número

Resultados parciais:  [0.746 0.773 0.751 0.762 0.756 0.759 0.756 0.753 0.759 0.763]
Média: 75.78%
Intervalo de acurácia: 74.37% ~ 77.19%


In [8]:
cv = KFold(n_splits = 10, shuffle = True) # Com embaralhamento

cross_validate_muda_params(9876, cv)

Resultados parciais:  [0.759 0.744 0.77  0.752 0.766 0.75  0.74  0.761 0.763 0.773]
Média: 75.78%
Intervalo de acurácia: 73.69% ~ 77.87%


# Simulação de uma situação ruim

Pode ser 'azar' ou uma proporção de exemplos desbalanceada.

Ex: Vamos ordenar o DF por vendido, dessa forma todos os carros vendidos estarão primeiro.

In [9]:
def imprime_resultados(resultados):
    resultados_teste = resultados['test_score']
    media_teste = resultados_teste.mean()
    desvio_padrao_teste = resultados_teste.std()
    intervalo_acuracia = [media_teste-(2*desvio_padrao_teste), media_teste+(2*desvio_padrao_teste)]  # [media - 2*desvio_padrao, media + 2*Desvio Padrao]

    print("Resultados parciais: ", resultados_teste)
    print("Média: %.2f%%" % (media_teste * 100))
    print("Intervalo de acurácia: %.2f%% ~ %.2f%%" % (intervalo_acuracia[0]*100, intervalo_acuracia[1]*100))

In [10]:
# Sort dos dados por 'vendido' e separação do x e y

dados_azar = dados.sort_values("vendido", ascending=True)

x_azar = dados_azar[['preco','idade_do_modelo','km_por_ano']]
y_azar = dados_azar['vendido']

In [11]:
modelo_azar = DecisionTreeClassifier(max_depth = 2)

cv = KFold(n_splits = 10) ## Sem embaralhar
resultados_azar = cross_validate(modelo_azar, x_azar, y_azar, cv = cv) # cv -> em quantos pedaços a validação será quebrada

imprime_resultados(resultados_azar)

# Resultados parciais:  [0.447 0.409 0.438 0.446 0.694 0.663 0.668 0.673 0.67  0.676]
# Média: 57.84%
# Intervalo de acurácia: 34.29% ~ 81.39%

Resultados parciais:  [0.447 0.409 0.438 0.446 0.694 0.663 0.668 0.673 0.67  0.676]
Média: 57.84%
Intervalo de acurácia: 34.29% ~ 81.39%


Como pode ser visto, a média de acurácia ficou em 57%, bem abaixo da anterior. Rodando novamente, mas com os dados embaralhados, voltamos a média normal:

In [12]:
modelo_azar = DecisionTreeClassifier(max_depth = 2)

cv = KFold(n_splits = 10, shuffle = True) ## Embaralhar
resultados_azar = cross_validate(modelo_azar, x_azar, y_azar, cv = cv) # cv -> em quantos pedaços a validação será quebrada

imprime_resultados(resultados_azar)

# Resultados parciais:  [0.746 0.759 0.745 0.747 0.766 0.777 0.749 0.74  0.78  0.769]
# Média: 75.78%
# Intervalo de acurácia: 73.05% ~ 78.51%

Resultados parciais:  [0.746 0.759 0.745 0.747 0.766 0.777 0.749 0.74  0.78  0.769]
Média: 75.78%
Intervalo de acurácia: 73.05% ~ 78.51%


Existe também o Stratified KFold, para manter as proporções entre os pedaços de dados.

- Se no total existem 70% de carros vendidos, nas partes também terá 70% de carros vendidos, e etc.
- Bastante usado quando existe muita diferença entre as classes de dados.

In [13]:
from sklearn.model_selection import StratifiedKFold

modelo_azar = DecisionTreeClassifier(max_depth = 2)

cv = StratifiedKFold(n_splits = 10, shuffle = True) ## Embaralhar
resultados_azar = cross_validate(modelo_azar, x_azar, y_azar, cv = cv) # cv -> em quantos pedaços a validação será quebrada

imprime_resultados(resultados_azar)

# Resultados parciais:  [0.75  0.766 0.762 0.745 0.744 0.742 0.753 0.783 0.759 0.772]
# Média: 75.76%
# Intervalo de acurácia: 73.22% ~ 78.30%

Resultados parciais:  [0.771 0.779 0.754 0.746 0.756 0.755 0.736 0.746 0.755 0.78 ]
Média: 75.78%
Intervalo de acurácia: 73.02% ~ 78.54%


No mundo real, dificilmente teremos uma situação em que um dado novo surge e é muito parecido com algum dado de teste. Por exemplo, os algoritmos KFold dividem os grupos de treino e teste estratificando as classes, ou seja, mantendo as proporções para cada grupo (% de vendidos é igual para teste e treino).

- Em resumo, dificilmente os dados do mundo real são estraficados e embaralhados.

Nesse contexto, podemos utilziar o Group KFold, que irá dividir por grupos. Esse algoritmo necessita saber qual feature será usada para dividir os grupos. Esse parâmetro é o groups do método cross_validate:

In [19]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth = 2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.idade_do_modelo)

imprime_resultados(resultados)

Resultados parciais:  [0.7586548  0.78389399 0.76394422 0.73635427 0.7560241  0.76626016
 0.73207171 0.76525336 0.7589545  0.75691134]
Média: 75.78%
Intervalo de acurácia: 72.97% ~ 78.59%


# Pipeline de pré-processamento e validação

Com modelos sensíveis a escala dos dados (como o SVC) temos que usar um scaler. Porém, todo split que o KFold fizer, deve ser processador os dados no scaler e rodada a validação novamente.

Isso pode ser feito através do Pipeline:

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

np.random.seed(301)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)]) ## Para cada split do KFold, o pipeline irá todar tanto o scaler quanto o estimador

cv = GroupKFold(n_splits = 10)
resultados = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.idade_do_modelo)

imprime_resultados(resultados)

Resultados parciais:  [0.76063304 0.77573904 0.78784861 0.73944387 0.76907631 0.7754065
 0.7689243  0.76525336 0.76089061 0.7607245 ]
Média: 76.64%
Intervalo de acurácia: 74.22% ~ 79.06%


# Conclusão

A média e o intervalo providos pela validação cruzada diz o quão bem você espera que o modelo se comporte com dados previamente desconhecidos mas... se você usou cross validation com 10 folds, qual dos 10 modelos treinados você vai usar agora no mundo real?

A ideia é que a validação cruzada num conjunto de dados **somente te diz o que você pode esperar. Ela não treina o seu modelo final**. Para isso, basta treinar o modelo com todos os dados normalmente:

modelo.fit(x, y)